In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio


from mainnet_launch.data_fetching.quotes.top_level_check_exit_liquidity import fetch_quotes
from mainnet_launch.data_fetching.quotes.get_all_underlying_reserves import fetch_raw_amounts_by_destination
from mainnet_launch.constants import *

pio.templates.default = None

# dola is broken
autopool = AUTO_ETH

chain = autopool.chain

block = chain.client.eth.block_number

lens_contract_df,reserve_token_ownership_df, token_df = fetch_raw_amounts_by_destination(block, chain)
raw_balance_by_token = reserve_token_ownership_df[reserve_token_ownership_df['autopool_symbol'] ==  autopool.symbol] .groupby('token_address')['reserve_amount'].sum()

balances_by_tokens = raw_balance_by_token.to_dict()
quote_df = await fetch_quotes(BASE_USD, balances_by_tokens)


2025-07-02 18:19:38.257 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


KeyError: '0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6'

In [ ]:
def compute_excess_slippage_from_size(quote_df: pd.DataFrame) -> pd.DataFrame:
    df = quote_df.groupby(["symbol", "sell_amount_norm"])["ratio"].first().reset_index()
    buy_token_ratio_at_sell_1_token = (
        df[df["sell_amount_norm"] == 1][["symbol", "ratio"]].set_index("symbol").to_dict()["ratio"]
    )
    highest_sold_amount = df.groupby("symbol")["sell_amount_norm"].max().to_dict()

    df["buy_token_ratio_at_one_token_sold"] = df["symbol"].map(buy_token_ratio_at_sell_1_token)
    df["highest_sold_amount"] = df["symbol"].map(highest_sold_amount)
    df["percent_sold"] = df.apply(lambda row: round(100 * row["sell_amount_norm"] / row["highest_sold_amount"], 2), axis=1)
    df["bps_excess_loss_vs_1"] = df.apply(
        lambda row: 10_000 * (row["buy_token_ratio_at_one_token_sold"] - row["ratio"]) / row["ratio"], axis=1
    )
    return df

slippage_df = compute_excess_slippage_from_size(quote_df)

px.scatter(slippage_df, x="percent_sold", y="bps_excess_loss_vs_1", color="symbol").show()
px.scatter(slippage_df, x="sell_amount_norm", y="bps_excess_loss_vs_1", color="symbol").show()

In [ ]:
# it gives you slippage on WETH -> WETH, 

# odds